In [ ]:
import numpy as np
import pandas as pd
import re
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [ ]:
df = pd.read_excel('/drive/My Drive/6SP/logs.xlsx')
df_desc = df[df['Description'].notnull()] # remove rows with empty description field

In [ ]:
def userId(desc):
    return desc.split()[4][1:-1]

In [ ]:
df_desc['User_id'] = df_desc['Description'].apply(userId)
#df_desc.head()

In [ ]:
df_new = df_desc[df_desc['Event context'].notnull()] # remove rows with empty event context field
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id
0,"28/02/22, 10:56",Course: GP106-2021 : Computing,Logs,Log report viewed,The user with id '2614' viewed the log report ...,web,2614
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web,2614
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web,3740
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30
4,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Role assigned,The user with id '30' assigned the role with i...,web,30


In [ ]:
# function to extract different event context types  
def extractEvent(eventContext):
  return eventContext.split(":")[0]

In [ ]:
df_new['Event'] = df_new['Event context'].apply(extractEvent)
#df_new.head()

In [ ]:
# extract only log records with event context type of "Label" and "File"
df_new = df_new[df_new.Event.isin(["Label","File"])]
#df_new.head()

In [ ]:
# function to extract different event context types  2nd value given
def extractEventName(eventContext):
  return eventContext.split(":")[1]

In [ ]:
df_new['EventName'] = df_new['Event context'].apply(extractEventName)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Event,EventName
14,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification
15,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...
16,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs
21,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs
24,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs


In [ ]:
# function to extract status from event name field
def extractStatus(eventName):
  return eventName.split()[2]

In [ ]:
df_new['Status'] = df_new['Event name'].apply(extractStatus)
#df_new.head()

In [ ]:
# function to give type from Event context field
def extractType(eventContext):
  if (eventContext.split(":")[1][1:8] == 'Lecture') & (len(eventContext.split(":")[1]) < 15) :
    return "Zoom_recording"
  else:
    return "File"

In [ ]:
df_new['Type'] = df_new['Event context'].apply(extractType)
#df_new.head()

In [ ]:
# function to extract module Id from description field
def extractModuleID(desc):
  if (desc.split()[5] == 'updated'):
    return desc.split()[-7]
  else:
    return desc.split()[-1]

In [ ]:
df_new['Module_id'] = df_new['Description'].apply(extractModuleID)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Event,EventName,Status,Type,Module_id
14,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,completion,File,'49119'
15,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,completion,File,'48654'
16,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,completion,File,'48652'
21,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,File,'34437'.
24,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,File,'34437'.


In [ ]:
df_new = df_new.set_index('Status')
df_new = df_new.rename(index={'completion':'completed'})
#df_new.head()

In [ ]:
df_new=df_new.reset_index()
#df_new.head()

In [ ]:
df_new.rename(columns={'EventName': 'Material_name', 'Time': 'Date_time'}, inplace=True)
df_new.head()

,Status,Date_time,Event context,Component,Event name,Description,Origin,User_id,Event,Material_name,Type,Module_id
0,completed,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,File,'49119'
1,completed,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,File,'48654'
2,completed,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,File,'48652'
3,viewed,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,File,'34437'.
4,viewed,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,File,'34437'.


In [ ]:
df_learningMaterials = df_new[['User_id','Material_name','Module_id','Type','Event','Status','Date_time']]
df_learningMaterials.head()

,User_id,Material_name,Module_id,Type,Event,Status,Date_time
0,3489,GP106 Project Specification,'49119',File,File,completed,"28/02/22, 06:59"
1,3489,GP106 Project\n\nInformation about the projec...,'48654',File,Label,completed,"28/02/22, 06:59"
2,3489,GP106 Labs,'48652',File,Label,completed,"28/02/22, 06:59"
3,3638,Group List for Labs,'34437'.,File,File,viewed,"28/02/22, 05:51"
4,3638,Group List for Labs,'34437'.,File,File,viewed,"28/02/22, 05:40"


In [ ]:
df_learningMaterials.to_csv("Learning_Materials.csv")